# CYP metabolizing phenotypes

"In mammals, __Cytochrome P450 (CYP)__ enzymes (...) are responsible for the oxidative metabolism of many xenobiotics as well as organic endogenous compounds. In humans, 57 isoforms were identified which are classified based on sequence homology" [\(Redlich et al. 2008\)](https://pubmed.ncbi.nlm.nih.gov/18828626/).

What are the frequencies of the __star allele__ haplotypes, and the observed
diplotypes and associated __metabolizing phenotypes__ of two isoforms -
CYP2C19 chromosome 10 and CYP2D6 chromosome 22 - in 1000 Genomes super-populations,
and UKB imputed and whole exome sequencing data?

<br>

In [1]:
import pandas as pd
import sqlite3
import vcf
import subprocess
import urllib.request
import json

from pathlib import Path
from itertools import compress, combinations, combinations_with_replacement

<br>

## Activity score, gene function, star allele, metabolizing phenotype

From this [meta-analysis](https://www.nature.com/articles/s41398-020-01129-1):

The __activity score (AS)__ system can be applied to both genes.

| AS | function |
|---|---|
| 0 | non-functional |
| 0.25 | decreased function |
| 0.5 | decreased function |
| 1 | normal function |
| 1.5 | increased function |

<br>

Corresponding __star alleles__

| AS | CYP2D6 | CYP2C19 |
|---|---|---|
| 0 | *3–*8, *15, *18, *31, *36, *47, *51, *56, *57, *62, *92, *100, *101 | *2, *3, *4, *5, *6, *7, *8, *23, *24 |
| 0.25–0.5 | *9, *10, *17, *29, *41, *49, *50, *54, *55, *59, *72 | *9, *10, *12, *16, *25, *27 |
| 1 | *1, *2, *27, *39, *45, *46, *48 | *1, *13, *15 *18 |
| 1.5 | *53 | *17 |
| Unknown |*43, *60, *65, *82, *84, *85, *86 | |

<br>

Associated diplotype __phenotypes__

PM = poor metabolizer, IM = intermediate metabolizer, NM = normal metabolizer,
RM = rapid metabolizer, UM = ultra-rapid metabolizer

<br>

| AS | phenotype | alleles |
|---|---|---|
| 0 | PM | homozygous carriers of non-functional alleles |
| 0.25-1 | IM | one functional or decreased function allele and one non-functional allele and those carrying two decreased function alleles |
| 1.25-2.25 | NM | homozygous carriers of normal function alleles and heterozygous carriers with one decreased function and one normal function allele |
| >2.25 | UM | one or more increased function alleles and carriers of a duplication or multiplication of a functional allele |

__Note.__ see the CPIC [Term Standardization for Clinical Pharmacogenetic Test Results Project](https://cpicpgx.org/resources/term-standardization/) and associated [publication](https://pubmed.ncbi.nlm.nih.gov/27441996/)

<br>

From [Dean 2017](https://www.ncbi.nlm.nih.gov/books/NBK425165/)

__CYP2C19 metabolizer phenotypes__

phenotype | e.g. diplotype | alleles |
|---|---|---|
| PM | *2/*2, *2/*3, *3/*3 | two non-functional alleles |
| IM | *1/*2, *1/*3, *2/*17? | one normal function and one no function allele or one no function allele and one increased function allele |
| NM | *1/*1 | two normal function alleles |
| RM | *1/*17 | one normal function allele and one increased function allele |
| UM | *17/*17 | two increased function alleles |


__Note__. There are $k$-$combination + n$ (i.e. $n$ choose $k$ _with repitition_)
possible diplotypes, where $k=2$ and $n$ is the number of star alleles.

<br>

## Variants used to call star alleles and phenotype

From PharmGKB [CYP2C19](https://www.pharmgkb.org/page/cyp2c19RefMaterials) and
[CYP2D6](https://www.pharmgkb.org/page/cyp2d6RefMaterials) reference material.

<br>

### CYP2C19 - chromosome 10


In [2]:
def display_df(data, nrow=6):
    """Display head and tail of dataframe"""
    with pd.option_context('display.max_rows', nrow):
        display(data)

CPIC CYP2C19 variant-star allele definition table

In [3]:
cyp2c19_variants = pd.read_excel('cpic/CYP2C19_allele_definition_table.xlsx', skiprows=1, header=None).transpose()
cyp2c19_variants = cyp2c19_variants.rename(columns=cyp2c19_variants.iloc[0]).drop(cyp2c19_variants.index[0])

display_df(cyp2c19_variants)

/users/k1342162/apps/miniconda3/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,NG_008384.3 (ATG start),Effect on protein (NP_000760.1),"Position at NC_000010.11 (Homo sapiens chromosome 10, GRCh38.p13)",Position at NG_008384.3 (CYP2C19 RefSeqGene; forward relative to chromosome),rsID,CYP2C19 Allele,*38,*1,*2,*3,...,*29,*30,*31,*32,*33,*34,*35,*36,*37,*39
1,-806C>T,NaN,g.94761900C>T,g.4220C>T,rs12248560,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1A>G,p.M1V,g.94762706A>G,g.5026A>G,rs28399504,NaN,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7C>T,p.P3S,g.94762712C>T,g.5032C>T,rs367543002,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,90080C>G,p.F448L,g.94852785C>G,g.95105C>G,rs118203759,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,90209A>C,p.X491C,g.94852914A>C,g.95234A>C,rs55640102,NaN,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Structural Variation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CYP2C19 full gene deletions,CYP2C19 partial gene deletions,NaN


<br>

CPIC CYP2C19 allele functionality table

In [4]:
cyp2c19_function = (pd.read_excel('cpic/CYP2C19_allele_functionality_reference.xlsx', skiprows=1)
                    .dropna(axis='columns', how='all'))
display_df(cyp2c19_function)

,Allele/cDNA/rsID,Allele Clinical Functional Status (Required),PMID (Required),Strength of Evidence (Required),Summary of Findings (Required)
0,*1,Normal function,"7487078, 32602114, 22027650",Definitve,CYP2C19*1 is assigned normal function based on...
1,*2,No function,"8195181, 22027650",Definitve,CYP2C19*2 is assigned no function based on def...
2,*3,No function,"7969038, 9103550, 22027650",Definitve,CYP2C19*3 is assigned no function based on def...
...,...,...,...,...,...
32,*36,No function,31260137,Limited,CYP2C19*36 is assigned no function based on li...
33,*37,No function,31260137,Limited,CYP2C19*37 is assigned no function based on li...
34,*38,Normal function,"2009263, 32602114",Definitve,CYP2C19*38 is assigned normal function based o...


In [5]:
cyp2c19_function['Allele Clinical Functional Status (Required)'].value_counts()

No function           9
Uncertain function    9
Normal function       7
Decreased function    6
 No function          3
Increased function    1
Name: Allele Clinical Functional Status (Required), dtype: int64

In [6]:
cyp2c19_function[cyp2c19_function['Allele/cDNA/rsID'].isin(['*17'])]

,Allele/cDNA/rsID,Allele Clinical Functional Status (Required),PMID (Required),Strength of Evidence (Required),Summary of Findings (Required)
16,*17,Increased function,"16413245, 17625515, 20083681",Strong,CYP2C19*17 is assigned increased function base...


<br>

CPIC CYP2C19 diplotype-phenotype table

In [7]:

cyp2c19_diplotype = (pd.read_excel('cpic/CYP2C19_Diplotype_Phenotype_Table.xlsx', skipfooter=4)
                     .dropna(axis='columns', how='all'))
display_df(cyp2c19_diplotype)

/users/k1342162/apps/miniconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,CYP2C19 Diplotype,CYP2C19 phenotype,EHR Priority Result Notation
0,*1/*2,CYP2C19 Intermediate Metabolizer,Abnormal/Priority/High Risk
1,*1/*22,CYP2C19 Intermediate Metabolizer,Abnormal/Priority/High Risk
2,*1/*24,CYP2C19 Intermediate Metabolizer,Abnormal/Priority/High Risk
...,...,...,...
627,*9/*32,Indeterminate,none
628,*9/*33,Indeterminate,none
629,*9/*34,Indeterminate,none


Not all mathematically possible combinations appear the CPIC diplotype table, e.g.,

In [8]:
possible_combinations = list(combinations_with_replacement(['*' + str(x) for x in list(range(1, 39 +1))], 2))
possible_combinations = ['/'.join(x) for x in possible_combinations]

print(
    len(list(set(possible_combinations) - set(cyp2c19_diplotype['CYP2C19 Diplotype']))),
    'possible CYP2C19 diplotypes not included in CPIC table')

150 possible CYP2C19 diplotypes not included in CPIC table


In [9]:
# Star alleles corresponding to metabolizing phenotypes

# ['*2', '*3', '*4', '*5', '*6', '*7', '*8', '*23', '*24']
# ['*9', '*10', '*12', '*16', '*25', '*27']
# cyp2c19_variants[['rsID', '*1', '*13', '*15', '*18']].dropna(how='all', subset=['*1', '*13', '*15', '*18'])

# Increased function
cyp2c19_variants[['rsID', 'Position at NC_000010.11 (Homo sapiens chromosome 10, GRCh38.p13)', '*17']].dropna(how='all', subset=['*17'])

,rsID,"Position at NC_000010.11 (Homo sapiens chromosome 10, GRCh38.p13)",*17
1,rs12248560,g.94761900C>T,T
27,rs3758581,g.94842866A>G,G


<br>

### CYP2D6 - chromosome 22

In [10]:
cyp2d6_variants = pd.read_excel('cpic/CYP2D6_allele_definition_table.xlsx', skiprows=1, header=None).transpose()
cyp2d6_variants = cyp2d6_variants.rename(columns=cyp2d6_variants.iloc[0]).drop(cyp2d6_variants.index[0])
display_df(cyp2d6_variants)

/users/k1342162/apps/miniconda3/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,NG_008376.4 (ATG start),Effect on protein (NP_000097.3),"Position at NC_000022.11 (Homo sapiens chromosome 22, GRCh38.p13)",Position at NG_008376.4 (CYP2D6 RefSeqGene; reverse relative to chromosome),rsID,CYP2D6 Allele,*1,*2,*3,*4,...,*139,*140,*141,*142,*143,*144,*145,*146,*147,*149
1,14C>T,p.A5V,g.42130778G>A,g.5033C>T,rs773790593,NaN,G,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19G>A,p.V7M,g.42130773C>T,g.5038G>A,rs72549358,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31G>A,p.V11M,g.42130761C>T,g.5050G>A,rs769258,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,4187C>T,p.S488F,g.42126605G>A,g.9206C>T,rs568495591,NaN,G,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,4214G>A,p.R497H,g.42126578C>T,g.9233G>A,rs1440526469,NaN,C,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,Structural Variation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CYP2D6 diplotype-phenotype table

In [11]:
cyp2d6_diplotype = (pd.read_excel('cpic/CYP2D6_Diplotype_Phenotype_Table.xlsx')
                     .dropna(axis='columns', how='all'))
display_df(cyp2d6_diplotype)

,CYP2D6 Diplotype,Activity Score,Coded Diplotype/Phenotype Summary,EHR Priority Notation
0,*1/*1,2,Normal Metabolizer,Normal/Routine/ Low Risk
1,*1/*1x2,3,Ultrarapid Metabolizer,Abnormal/Priority/High Risk
2,*1/*1≥3,≥4,Ultrarapid Metabolizer,Abnormal/Priority/High Risk
...,...,...,...,...
11023,*138/*138,NaN,Indeterminate,none
11024,*138/*139,NaN,Indeterminate,none
11025,*139/*139,NaN,Indeterminate,none


<br>

CPIC allele functionality table

In [12]:
cyp2d6_function = (pd.read_excel('cpic/CYP2D6_allele_functionality_reference.xlsx', skiprows=1)
                    .dropna(axis='columns', how='all'))
display_df(cyp2d6_function)

,,Activity Value (Optional),Allele Clinical Function Status (Required)*,PMID (Optional),Strength of Evidence (Optional),Findings (Optional),Comments
0,*1,1.0,Normal function,2574001,NaN,2574001: debrisoquine (in vivo),NaN
1,*1x2,2.0,Increased function,7616439; 9012401; 17259947; 19541866; 22111604...,NaN,7616439: debrisoquine (in vivo); 9012401: dext...,NaN
2,*1≥3,≥3.0,Increased function,7616439; 9012401; 17259947; 19541866; 22111604...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 'Allele/cDNA/rsID'
# cyp2d6_function.columns
cyp2d6_function[cyp2d6_function['  '].isin(['*5'])]

,,Activity Value (Optional),Allele Clinical Function Status (Required)*,PMID (Optional),Strength of Evidence (Optional),Findings (Optional),Comments
10,*5,0,No function,1673290; 11266079,NaN,"1673290: debrisoquine, sparteine (in vivo); 11...",NaN


### Variants in UKB imputed genetic data

In [9]:
def rsid_count(data, chromosome, gene):
    """Count variants in CPIC reference and imputed data

    data (pd.DataFrame): transposed CPIC defintion table as dataframe
    chromosome (int): chromosome number
    gene (str): PGx gene
    """
    rsids = list(compress(
        data['rsID'].dropna(),
        data['rsID'].dropna().str.startswith('rs')
        ))
    
    print(len(rsids), f'rsID SNPs used to call {gene} star alleles')
    
    bgen_file_list = [
        f'/scratch/datasets/ukbiobank/June2017/Imputed/ukb_imp_chr{chromosome}_v3_MAF1_INFO4.bgen.bgi',
        f'/scratch/datasets/ukbiobank/June2017/Imputed/ukb_imp_chr{chromosome}_v3_MAF0_INFO7.bgen.bgi',
        f'/scratch/datasets/ukbiobank/June2017/Imputed/ukb_imp_chr{chromosome}_v3.bgen.bgi']
    
    for f in bgen_file_list:
        file_name = Path(f).name
        con = sqlite3.connect(f)
        bgi = pd.read_sql_query('SELECT * FROM Variant', con)
        print(bgi[bgi['rsid'].isin(rsids)].shape[0], f'of {len(rsids)} included in {file_name}')
        con.close()

In [10]:
rsid_count(cyp2c19_variants, 10, 'CYP2C19')

34 rsID SNPs used to call CYP2C19 star alleles
4 of 34 included in ukb_imp_chr10_v3_MAF1_INFO4.bgen.bgi
16 of 34 included in ukb_imp_chr10_v3_MAF0_INFO7.bgen.bgi
19 of 34 included in ukb_imp_chr10_v3.bgen.bgi


Additionally full deletion corresponds to *36; partial deletion to *37

In [ ]:
cyp2c19_variants.loc[cyp2c19_variants['rsID'].isna(), ]

<br>

Chromosome 22 variants used to call CYP2D6 star alleles

In [ ]:
rsid_count(cyp2d6_variants, 22, 'CYP2D6')

In [ ]:
cyp2d6_variants.loc[cyp2d6_variants['rsID'].isna(), ].dropna(axis='columns', how='all')

<br>

Write to file the list of rsID variants observed in the unfiltered UKB data. Note. this does not includes indels - check how these are encoded captured in the VCF file.

In [ ]:

def write_rsids(csv_file, data, chromosome):
    """Write to file the rsIDs observed in the unfiltered UKB imputed data"""
    rsids = list(compress(
        data['rsID'].dropna(),
        data['rsID'].dropna().str.startswith('rs')
        ))
    
    path = f'/scratch/datasets/ukbiobank/June2017/Imputed/ukb_imp_chr{chromosome}_v3.bgen.bgi'
    con = sqlite3.connect(path)
    bgi = pd.read_sql_query('SELECT * FROM Variant', con)
    bgi[bgi['rsid'].isin(rsids)]['rsid'].to_csv(csv_file, header=False, index=False)
    con.close()

In [ ]:
write_rsids('cyp2c19_rsids.csv', cyp2c19_variants, 10)
write_rsids('cyp2d6_rsids.csv', cyp2d6_variants, 22)

# Full list from PharmGKB
# cyp2c19_variants['rsID'].dropna().to_csv('cyp2c19_rsids.csv', header=False, index=False)
# cyp2d6_variants['rsID'].dropna().to_csv('cyp2d6_rsids.csv', header=False, index=False)